In [130]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingRegressor, StackingCVRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
import xgboost as xgb
from xgboost import XGBRegressor
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import sklearn.preprocessing as preprocessing

In [131]:
trainX = pd.read_csv('train_values.csv')
trainY = pd.read_csv('train_labels.csv')
testX = pd.read_csv('test_values.csv') 

In [132]:
#trainX.area__rucc = trainX.area__rucc.apply(lambda x: x.split(" -")[0])

In [133]:
trainX.area__rucc = trainX.area__rucc.astype("category").cat.codes
trainX.area__urban_influence = trainX.area__urban_influence.astype("category").cat.codes
trainX.econ__economic_typology = trainX.econ__economic_typology.astype("category").cat.codes
trainX.yr = trainX.yr.astype("category").cat.codes

In [134]:
# trainX = trainX.join(pd.get_dummies(data=trainX.area__rucc))
# trainX = trainX.join(pd.get_dummies(data=trainX.area__urban_influence))
# trainX = trainX.join(pd.get_dummies(data=trainX.econ__economic_typology))
# trainX.yr = trainX.yr.astype("category").cat.codes
# trainX.drop(['area__rucc', 'area__urban_influence', 'econ__economic_typology'], axis=1, inplace=True)

In [135]:
for col in trainX.columns:
    #trainX[col].fillna(trainX[col].mean(), inplace=True)
    trainX[col].fillna(trainX[col].mean(), inplace=True)

In [136]:
# for col in trainX.columns:
#     #trainX[col].fillna(trainX[col].mean(), inplace=True)
#     trainX[col].fillna(0, inplace=True)

In [137]:
#NAN values finder
trainX[pd.isnull(trainX).any(axis=1)]

,row_id,area__rucc,area__urban_influence,econ__economic_typology,econ__pct_civilian_labor,econ__pct_unemployment,econ__pct_uninsured_adults,econ__pct_uninsured_children,demo__pct_female,demo__pct_below_18_years_of_age,...,health__pct_diabetes,health__pct_low_birthweight,health__pct_excessive_drinking,health__pct_physical_inacticity,health__air_pollution_particulate_matter,health__homicides_per_100k,health__motor_vehicle_crash_deaths_per_100k,health__pop_per_dentist,health__pop_per_primary_care_physician,yr


In [138]:
trainX.drop('row_id', axis=1, inplace=True)
trainY.drop('row_id', axis=1, inplace=True)

In [139]:
# Feature Selection

# trainX.drop(['yr',
#  'health__air_pollution_particulate_matter',
#  'health__homicides_per_100k',
#  'demo__birth_rate_per_1k',
#  'demo__death_rate_per_1k'], axis=1, inplace=True)

In [140]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(trainX, trainY, test_size = 0.25, random_state = 42)

In [141]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (2398, 33)
Training Labels Shape: (2398, 1)
Testing Features Shape: (800, 33)
Testing Labels Shape: (800, 1)


# Standardization

In [142]:
# scaler = preprocessing.StandardScaler()
# scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.MaxAbsScaler()
# scaler = preprocessing.Normalizer()

# scaler.fit(train_features)
# train_features = scaler.transform(train_features)
# test_features = scaler.transform(test_features)

# Model

In [143]:
# dtr/ada params from submission 8 - RMSE : 3.4678
adaDTR_adaParams8 = {'n_estimators':400, 
                     'random_state':736283, 
                     'learning_rate':1}

adaDTR_dtrParams = {'max_depth':None, 
                    'min_samples_split':6, 
                    'max_features':'auto'}

#etr params from submission 13 - RMSE : 3.2440
etrParams = {'n_estimators':600, 
             'random_state':736283, 
             'max_depth':None, 
             'max_features':'auto', 
             'min_samples_split':2, 
             'n_jobs':-1}

#xgb params from submission 22 - RMSE : 3.3746
xgbParams = {'learning_rate':0.024, 
             'n_estimators':18000, 
             'max_depth':6, 
             'min_child_weight':1,
             'gamma':0,
             'subsample':0.7,
             'colsample_bytree':0.6,
             'reg_alpha':1,
             'scale_pos_weight':1,
             'random_state':736283, 'n_jobs':-1}

rfgParams = {'n_estimators':390,
             'max_features':'auto',
             'min_samples_leaf':1,
             'max_depth':None,
             'random_state':736283,
             'n_jobs':-1}

etParams = {'criterion':'mse',
            'max_depth':12,
            'max_features':'auto',
            'min_samples_split':2,
            'min_samples_leaf':1,
            'random_state':736283}

adaParams = {'n_estimators':500, 'random_state':736283, 'learning_rate':1}

gbrParams = {'n_estimators':800, 
             'random_state':736283,
             'max_depth':8,
             'criterion':'friedman_mse',
             'max_features':'auto',
             'min_samples_split':3
             }

lgbmParams = {'boosting_type':'gbdt',
               'random_state':736283,
               'num_leaves':400,
               'min_data_in_leaf':5,
               'max_bin':500,
               'learning_rate':0.08,
               'n_estimators':300,
               'n_jobs':-1,
               'subsample':0.8
        }

forestParams = {'bootstrap': True,
             'max_depth': 70,
             'max_features': 'auto',
             'min_samples_leaf': 4,
             'min_samples_split': 10,
             'n_estimators': 1000}

In [144]:
def build_StackingModel_complex():
    adaModel = AdaBoostRegressor(DecisionTreeRegressor(**adaDTR_dtrParams), **adaDTR_adaParams8)
    etrModel = ExtraTreesRegressor(**etrParams)
    xgbModel = XGBRegressor(**xgbParams)
    rfgModel = RandomForestRegressor(**rfgParams)
    gbrModel = GradientBoostingRegressor(**gbrParams)
    adaETRModel = AdaBoostRegressor(ExtraTreeRegressor(**etParams),**adaParams)
    lgbmModel = LGBMRegressor(**lgbmParams)
    
    forest = RandomForestRegressor(**forestParams)
    
    stregr = StackingRegressor(regressors=[adaModel, etrModel, gbrModel, adaETRModel, lgbmModel, xgbModel], meta_regressor=forest)
    
    return stregr

In [151]:
def build_StackingModel():
    adaModel = AdaBoostRegressor(DecisionTreeRegressor(),n_estimators=100)
    etrModel = ExtraTreesRegressor(n_estimators=100)
    gbrModel = GradientBoostingRegressor(n_estimators=100)
    adaETRModel = AdaBoostRegressor(ExtraTreeRegressor(),n_estimators=100)
    lgb = LGBMRegressor(learning_rate=0.8,n_estimators=100) #8000
    xgb =XGBRegressor(n_estimators=18000) #18000 rmse 88
#     xgb = XGBRegressor(max_depth=10, n_estimators=18000, learning_rate=0.1) #best score
#     xgb = XGBRegressor(max_depth=5, min_child_weight=5, n_estimators=18000, learning_rate=0.1, subsample=0.8)
    # Instantiate model with 1000 decision trees
    ridge = Ridge(random_state=1)
    lasso = Lasso(random_state=1)
    svr_rbf = SVR(kernel='rbf')
    forest = RandomForestRegressor(n_estimators = 1000, random_state = 42)
#     forest = RandomForestRegressor(**forestParams)
    neural = MLPRegressor(hidden_layer_sizes=(100,10000,100),max_iter=500)
    
    # current best score
    stregr = StackingRegressor(regressors=[adaModel, etrModel, gbrModel, adaETRModel, lgb, xgb], meta_regressor=forest)
    
    return stregr

In [146]:
from sklearn.grid_search import GridSearchCV

def xgb_GridSearchCV():
    cv_params = {'learning_rate': [0.1, 0.01], 'subsample': [0.7,0.8,0.9]}
    xgb = XGBRegressor(max_depth=5, min_child_weight=5, n_estimators=1000, learning_rate=0.1, subsample=0.8)
    optimized_GBM = GridSearchCV(xgb, 
                            cv_params,
                                scoring='r2', cv = 5, n_jobs = -1) 
    return optimized_GBM

#'max_depth': 5, 'min_child_weight': 5
#learning_rate': 0.1, 'subsample': 0.8

In [147]:
def xgb_optimize():
    train_xgdmat = xgb.DMatrix(train_features, train_labels.iloc[:, 0])
    test_xgdmat = xgb.DMatrix(test_features, test_labels.iloc[:, 0])
    params = {'max_depth':5, 
              'min_child_weight':5, 
              'n_estimators':1000, 
              'learning_rate':0.1,
              'subsample':0.8,
              'objective':'reg:linear'}
    
#     model = xgb.train(params, 
#                     train_xgdmat, 
#                     num_boost_round = 3000, 
#                     evals=[(test_xgdmat, "Test")],
#                     early_stopping_rounds = 100)
    
    model = xgb.train(params, 
                    train_xgdmat, 
                    num_boost_round = 2370)
    return model
    
#2370

In [148]:
model = build_StackingModel()
# model = xgb_with_optimization()
# model = xgb_optimize()
# test_xgdmat = xgb.DMatrix(test_features)
# pred = model.predict(test_xgdmat)

In [149]:
model.fit(train_features, train_labels.iloc[:, 0])
pred = model.predict(test_features)

In [150]:
mse = mean_squared_error(test_labels.iloc[:, 0], pred)

print("RMSE: %.4f\n" % sqrt(mse))
# 28.40
# standard 28.3863
# minmax 28.4783
# maxabs 28.2943 with neural 28.4779
# normal 30.1721
# print(model.grid_scores_)


RMSE: 28.5751



# Feature Importance

In [152]:
# importances = model.feature_importances_
# indices = np.argsort(importances)[::-1]

# plt.title("Feature importances")
# plt.bar(range(train_features.shape[1]), importances[indices],
#        color="r", align="center")
# plt.xticks(range(train_features.shape[1]), indices)
# plt.xlim([-1, train_features.shape[1]])
# plt.show()

# Test for capstone

In [153]:
testX = pd.read_csv('test_values.csv')

test_row_id = testX.row_id
testX.drop('row_id', axis=1, inplace=True)

In [154]:
testX.area__rucc = testX.area__rucc.astype("category").cat.codes
testX.area__urban_influence = testX.area__urban_influence.astype("category").cat.codes
testX.econ__economic_typology = testX.econ__economic_typology.astype("category").cat.codes
testX.yr = testX.yr.astype("category").cat.codes

In [155]:
# testX = testX.join(pd.get_dummies(data=testX.area__rucc))
# testX = testX.join(pd.get_dummies(data=testX.area__urban_influence))
# testX = testX.join(pd.get_dummies(data=testX.econ__economic_typology))
# testX.yr = testX.yr.astype("category").cat.codes
# testX.drop(['area__rucc', 'area__urban_influence', 'econ__economic_typology'], axis=1, inplace=True)

In [156]:
for col in testX.columns:
    #trainX[col].fillna(trainX[col].mean(), inplace=True)
    testX[col].fillna(testX[col].mean(), inplace=True)

In [157]:
# for col in testX.columns:
#     #trainX[col].fillna(trainX[col].mean(), inplace=True)
#     testX[col].fillna(testX[0, inplace=True)

In [158]:
testX[pd.isnull(testX).any(axis=1)]

,area__rucc,area__urban_influence,econ__economic_typology,econ__pct_civilian_labor,econ__pct_unemployment,econ__pct_uninsured_adults,econ__pct_uninsured_children,demo__pct_female,demo__pct_below_18_years_of_age,demo__pct_aged_65_years_and_older,...,health__pct_diabetes,health__pct_low_birthweight,health__pct_excessive_drinking,health__pct_physical_inacticity,health__air_pollution_particulate_matter,health__homicides_per_100k,health__motor_vehicle_crash_deaths_per_100k,health__pop_per_dentist,health__pop_per_primary_care_physician,yr


# Standardization

In [159]:
# scaler = preprocessing.StandardScaler()
# scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.MaxAbsScaler()
# scaler = preprocessing.Normalizer()

# scaler.fit(trainX)
# trainX = scaler.transform(trainX)
# testX = scaler.transform(testX)

## Feature Selection

In [160]:
# testX.drop(['yr',
#  'health__air_pollution_particulate_matter',
#  'health__homicides_per_100k',
#  'demo__birth_rate_per_1k',
#  'demo__death_rate_per_1k'], axis=1, inplace=True)

In [161]:
model = build_StackingModel()
model.fit(trainX, trainY.iloc[:, 0])
pred = model.predict(testX)

In [162]:
pred.shape

(3080,)

In [163]:
testX.shape

(3080, 33)

In [164]:
test_row_id = pd.DataFrame({'row_id': test_row_id})
predictions = pd.DataFrame({'heart_disease_mortality_per_100k': pred})
final_prediction = test_row_id.join(predictions)
final_prediction.heart_disease_mortality_per_100k = predictions.heart_disease_mortality_per_100k.astype(int)
final_prediction.to_csv('submission.csv', index=False)